# Original Jeepney Routes - Fitness Function

## Route Connection and Genetic Algorithm Functions

### Setting Up

In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import networkx as nx
import shapely
import folium
import geojson
import math
import osmnx as ox
from rtree import index as rtree_index
import pickle
import copy
import utm

from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon, LineString, Point
from geopy.distance import geodesic
from shapely.ops import split

from __future__ import absolute_import, division
from math import radians, sin, cos, sqrt, atan2, exp, log
import webbrowser
import random
from scipy.spatial import KDTree
from scipy.spatial.distance import euclidean

ox.settings.log_console=True
ox.settings.use_cache=True

In [72]:
# Defining classes for the dataframes      
class stopCandidate:
    def __init__(self, lat, long, isTranspo, id):
        self.lat = lat
        self.long = long
        self.isTranspo = isTranspo
        self.id = id #Stop ID
        
    def getLat(self):
        return self.lat
    
    def getLong(self):
        return self.long
    
    def getDegree(self):
        return self.degree
    
class networkObj:
    def __init__(self, routes, stops, graph):
        self.routes = routes
        self.stops = stops
        self.fitness_score = 0
        self.graph = graph
        
class Route:
    def __init__(self, route, route_id):
        self.route = route
        self.route_id = route_id

In [73]:
# For units
def degrees_to_meters(angle_degrees):
    return angle_degrees * 6371000 * math.pi / 180

def meters_to_degrees(distance_meters):
    return distance_meters / 6371000 * 180 / math.pi

In [74]:
# Import and Export networks or graphs to pickle
def export_networks(networks, path):
    with open(path, 'wb') as f:
        pickle.dump(networks, f)

def import_networks(path):
    with open(path, 'rb') as f:
        routes = pickle.load(f)
    return routes

#### Graph and Features

In [75]:
# GENERATION OF MAIN CITY GRAPH
# IF FIRST TIME RUNNING, RUN THIS CODE TO GENERATE THE GRAPH
def generate_graph():
    mode = 'drive'
    graph = ox.graph_from_place(select_city, network_type = mode) # Generate graph of Metro manila
    ox.save_graphml(graph, city_file) # Save it as a file

def load_graph():
    graph = ox.load_graphml(city_file)
    
    print("Graph loaded successfully")
    print("NUMBER OF EDGES: ", graph.number_of_edges())
    print("NUMBER OF NODES: ", graph.number_of_nodes())
    print('\n')
    return graph

select_city = "Metro Manila, Philippines"
city_file = 'map/Metro Manila.graphml'
#generate_graph()
METROMANILA_GRAPH = load_graph()

select_city = "Manila, Philippines"
city_file = 'map/Manila.graphml'
#generate_graph()
MANILA_GRAPH = load_graph()

select_city = "Mandaluyong, Philippines"
city_file = 'map/Mandaluyong.graphml'
#generate_graph()
MANDALUYONG_GRAPH = load_graph()

select_city = "Makati, Philippines"
city_file = 'map/Makati.graphml'
#generate_graph()
MAKATI_GRAPH = load_graph()

Graph loaded successfully
NUMBER OF EDGES:  149172
NUMBER OF NODES:  59386


Graph loaded successfully
NUMBER OF EDGES:  12617
NUMBER OF NODES:  4926


Graph loaded successfully
NUMBER OF EDGES:  2374
NUMBER OF NODES:  989


Graph loaded successfully
NUMBER OF EDGES:  5505
NUMBER OF NODES:  2270




### Reading Data

In [76]:
# Reading all points

manila_points_gdf = gpd.read_file('./City Data/Manila_point.geojson')
mandaluyong_points_gdf = gpd.read_file('./City Data/Mandaluyong_point.geojson')
makati_points_gdf = gpd.read_file('./City Data/Makati_point.geojson')

manila_transpo_gdf = manila_points_gdf[manila_points_gdf['amenity'] == 'transportation']
mandaluyong_transpo_gdf = mandaluyong_points_gdf[mandaluyong_points_gdf['amenity'] == 'transportation']
makati_transpo_gdf = makati_points_gdf[makati_points_gdf['amenity'] == 'transportation']

In [102]:
# Checks if point is out of the specific geographic location
def remove_if_out_of_bounds(city_graph, routes):
    nodes = ox.graph_to_gdfs(city_graph, edges=False)
    min_x, min_y, max_x, max_y = nodes.total_bounds
    
    new_routes = []
    for route in routes:
        new_route = []
        for point in route:
            lat = point[0]
            long = point[1]
            
            if min_x <= long <= max_x and min_y <= lat <= max_y:
                new_route.append(point)
        new_routes.append(new_route)
            
    return new_routes

In [77]:
# Index for optimization
manila_idx = rtree_index.Index()
for i, row in manila_transpo_gdf.iterrows():
    manila_idx.insert(i, row['geometry'].bounds)
    
mandaluyong_idx = rtree_index.Index()
for i, row in mandaluyong_transpo_gdf.iterrows():
    mandaluyong_idx.insert(i, row['geometry'].bounds)
    
makati_idx = rtree_index.Index()
for i, row in makati_transpo_gdf.iterrows():
    makati_idx.insert(i, row['geometry'].bounds)
    

In [78]:
# Function to Extract all the stop points per row
def extract_points(df):
    routes = []
    columns = list(df.columns)
    
    for index, row in df.iterrows():
        points = []
        for col in columns:
            cell_value = row[col]
            if pd.isna(cell_value):
                continue
                
            if cell_value.endswith(']'):
                cell_value = cell_value[:-1] + ',' # Remove the last bracket if it's there
                
            data = eval(cell_value)[0]
            
            lat = data[0]
            lon = data[1]
            points.append((lat, lon))
        routes.append(points)
    return routes

In [79]:
# Reading the original jeepney route data
manila_df = pd.read_csv('Original Jeepney Routes/Original Data/Manila Routes.csv')
makati_df = pd.read_csv('Original Jeepney Routes/Original Data/Makati Routes.csv')
mandaluyong_df = pd.read_csv('Original Jeepney Routes/Original Data/Mandaluyong Routes.csv')

manila_route_stops = extract_points(manila_df)
makati_route_stops = extract_points(makati_df)
mandaluyong_route_stops = extract_points(mandaluyong_df)

### Stop Placement

In [80]:
def get_close_points(idx, point, max_distance=100):
    close_points = [i for i in idx.intersection((point.x - max_distance, point.y - max_distance, point.x + max_distance, point.y + max_distance))]
    return close_points

def check_transpo_stops(routes, point_gdf, city_graph, city_idx):
    transpo_points = []
    non_transpo_points = []
    new_route_list_with_stop_id = []
    
    stop_id = 0
    for route in routes:
        new_route = []
        for point in route:
            lat = point[0]
            long = point[1]
            isTranspo = False
            
            close_points = get_close_points(city_idx, Point(long, lat))
            for i, row in point_gdf.loc[close_points].iterrows():
                distance_to_point = geodesic(point, (row['y'], row['x'])).meters
                
                # Check if distance is within 100 meters and it's a transportation amenity
                if distance_to_point <= 100 and row['amenity'] == 'transportation':
                    isTranspo = True
                    break
                
            new_stop = stopCandidate(lat, long, isTranspo, stop_id)
            new_route.append(new_stop)
            
            if isTranspo:
                transpo_points.append(new_stop)
            else:
                non_transpo_points.append(new_stop)
            stop_id += 1
        
        new_route_list_with_stop_id.append(new_route)
        
    return transpo_points, non_transpo_points, new_route_list_with_stop_id
        
        

In [81]:
# x - lon - 120
# y - lat - 14
# LineString(lon, lat)
# geodesic(lat, lon)

def add_point_to_graph(point, graph_of_stops, list_of_stops, is_transpo):
    lat = point.lat
    long = point.long
    # Find the nearest edge to the location point
    nearest_edge = ox.distance.nearest_edges(graph_of_stops, X=long, Y=lat, return_dist=False)
    node1, node2, key = nearest_edge
    edge = graph_of_stops.get_edge_data(node1, node2)
    data = list(edge.values())[0]
        
    # get the line
    if 'geometry' not in data:
        gdf_edges = ox.graph_to_gdfs(graph_of_stops, nodes=False, edges=True)
        line = gdf_edges.loc[nearest_edge]['geometry']
        edge_length = gdf_edges.loc[nearest_edge]['length']
    else:
        line = data['geometry']
        edge_length = data['length']
        
        point_a = (graph_of_stops.nodes[node1]['y'], graph_of_stops.nodes[node1]['x'])
        point_c = (graph_of_stops.nodes[node2]['y'], graph_of_stops.nodes[node2]['x']) 
        
        if len(line.coords) == 2:
            # Get a random position between the line
            position_est = geodesic(point_a, (lat, long)).meters / edge_length
            
            if position_est > 1:
                point_position = random.uniform(0.4 * edge_length, edge_length * 0.6)
            else:
                point_position = random.uniform(position_est - 0.1, position_est + 0.1)
                
            # Convert to UTM
            node1_utm = utm.from_latlon(point_a[0], point_a[1])
            node2_utm = utm.from_latlon(point_c[0], point_c[1])

            utm_line = LineString([(node1_utm[0], node1_utm[1]), (node2_utm[0], node2_utm[1])])

            # Calculate the coordinate along the edge at the random position
            point_on_road_utm = calculate_coordinate_along_edge(utm_line, point_position)
            to_convert = (point_on_road_utm[0], point_on_road_utm[1], node1_utm[2], node1_utm[3])
            point_on_road = utm.to_latlon(*to_convert)
            lon = point_on_road[1]
            lat = point_on_road[0]
        else:
            # Accessing all coordinates
            coordinates = list(line.coords)
            coordinates = coordinates[1:-1]
            # Randomly pick one coordinate
            point_on_road = random.choice(coordinates)
            lon = point_on_road[0]
            lat = point_on_road[1]
        
        point.lat = lat
        point.long = lon
        graph_of_stops.add_node(point.id, x=lon, y=lat, isTranspo=is_transpo)
        list_of_stops.append(point)
    
        # Get the points
        point_b = (lat, lon) # y,x
        
        # Calculate new geometries
        if len(line.coords) == 2:
            line_ab = LineString([(point_a[1], point_a[0]), (point_b[1], point_b[0])])
            line_bc = LineString([(point_b[1], point_b[0]), (point_c[1], point_c[0])])
        else:
            result = split(line, Point(point_b[1], point_b[0]))
            new_lines = [geom for geom in result.geoms]
            line_ab = new_lines[0]
            line_bc = new_lines[1]
        
        # Calculate the new distances
        distance_ab = geodesic(point_a, point_b).meters
        distance_bc = geodesic(point_b, point_c).meters
            
        # Get the edge data and adjust distances
        edge_data = data.copy()
        edge_data['length'] = distance_ab
        edge_data['geometry'] = line_ab
        graph_of_stops.add_edge(node1, point.id, **edge_data)
        
        edge_data['length'] = distance_bc
        edge_data['geometry'] = line_bc
        graph_of_stops.add_edge(point.id, node2, **edge_data)

        # Remove the original edge
        graph_of_stops.remove_edge(node1, node2)
        
def calculate_coordinate_along_edge(edge, position):
    # Calculate the coordinate along the edge at the given position
    point = edge.interpolate(position)
    return point.x, point.y

In [82]:
# CREATING STOPS
# It should return a list of coordinates/nodes for stop and a graph of stops
# if residential area, check if the population density

def place_stops_on_roads(transpo_points, non_transpo_points, graph_of_stops, list_of_stops):
    
    for point in transpo_points:
        add_point_to_graph(point, graph_of_stops, list_of_stops, True)
        
    for point in non_transpo_points:
        add_point_to_graph(point, graph_of_stops, list_of_stops, False)

### Route Network Generation

In [83]:
def generate_route_network(routes, city_graph):
    overall_graph = nx.MultiDiGraph() # The route network graph
    route_network = []
    num_routes = 0 # Count number of routes
    
    for route in routes:
        route_list = []
        prev_stop = None
        for stop in route:
            print(stop.id)
            
            if prev_stop == None:
                prev_stop = stop
                continue

            path = nx.shortest_path(city_graph, prev_stop.id, stop.id)
            route_list.append(path)
            
            route_id = f'{num_routes}-A' # This will be used as a key for the edge
            overall_graph.add_node(prev_stop.id, lat=prev_stop.lat, lon=prev_stop.long, isTranspo=prev_stop.isTranspo) # The origin
            overall_graph.add_node(stop.id, lat=stop.lat, lon=stop.long, isTranspo=stop.isTranspo) # The dest
            overall_graph.add_edge(prev_stop.id, stop.id, key=route_id, road_path = path) # Add edge
            
        route_network.append(route_list)
        num_routes += 1
               
    return route_network, overall_graph
        
        

### Fitness Function

In [84]:
# Fitness function

def compute_fitness_score(road_snapped_network_graph, num_failure_removal,
                          weight_random_failure, weight_targeted_failure, weight_connectivity):

    random_failure_robustness = compute_random_failure_robustness(road_snapped_network_graph, num_failure_removal)
    weighted_random_failure_robustness = weight_random_failure * random_failure_robustness

    targeted_failure_robustness = compute_targeted_failure_robustness(road_snapped_network_graph, num_failure_removal)
    weighted_targeted_failure_robustness = weight_targeted_failure * targeted_failure_robustness

    connectivity_score = compute_connectivity(road_snapped_network_graph)
    weighted_connectivity = weight_connectivity * connectivity_score
    
    # print("Random Failure Score: ", weighted_random_failure_robustness)
    # print("Target Failure Score: ", weighted_targeted_failure_robustness)
    # print("Connectivity: ", weighted_connectivity)

    # Will use this return for now to utilize target and random failure nodes 
    return weighted_connectivity - weighted_random_failure_robustness - weighted_targeted_failure_robustness
    # return weighted_radius_of_gyration

def compute_connectivity(network):
    # External connectivity - measure how connected is the jeepney route network with other modes of transpo
    
    # Get the ratio of transportation stops to total stops in the network
    transpo_stops = [node for node, node_data in network.nodes(data=True) if node_data['isTranspo'] == True]
    total_stops = len(network.nodes(data=True))
    transpo_stop_ratio = len(transpo_stops) / total_stops

    # Get the average degree of all transportation stops in the network
    if len(transpo_stops) > 0:
        avg_transpo_degree = sum(network.degree(stop) for stop in transpo_stops) / len(transpo_stops)
    else:
        avg_transpo_degree = 1

    # Find a way to normalize the two values and combine them 

    # Internal connectivity - measure how connected is each jeepney route to other jeepney routes
                    
    # This counts how many nodes have intersections (Meaning node is connected to more than one route by route ID)
    num_intersections = 0
    for node, node_data in network.nodes(data=True):
        if network.in_degree(node) > 1 or network.out_degree(node) > 1:
            num_intersections += 1

    
    # Change these weights based on what the expected values for 
    # the transpo_stop_ratio, avg_transpo_degree, and num_intersections will be
    external_weight = 0.5
    internal_weight = 0.5
    
    # TODO: Delete this
    # print("Transpo stop ratio: ", transpo_stop_ratio)
    # print("Num intersections: ", num_intersections)
    # print("Average degree: ", avg_transpo_degree)

    # Formula subject to change
    return external_weight * (transpo_stop_ratio * avg_transpo_degree) + internal_weight * num_intersections


def compute_random_failure_robustness(road_snapped_network_graph, num_removals):
    graph_copy = road_snapped_network_graph.copy() # Make a copy
    
    for i in range(num_removals):
        selected_node = random.choice(list(graph_copy.nodes()))
        graph_copy.remove_node(selected_node)

    diameter, avg_path_length = compute_network_statistics(graph_copy)
    return compute_failure_robustness(graph_copy, diameter)

def compute_targeted_failure_robustness(road_snapped_network_graph, num_removals):
    graph_copy = road_snapped_network_graph.copy() # Make a copy
    
    for i in range(num_removals):
        node_degrees = graph_copy.degree()
        # Iterate over the DegreeView object to find the maximum degree
        max_degree = max(degree for _, degree in node_degrees)
        max_degree_node = get_node_with_degree(node_degrees, max_degree)
        graph_copy.remove_node(max_degree_node)

    diameter, avg_path_length = compute_network_statistics(graph_copy)
    return compute_failure_robustness(graph_copy, diameter)

def compute_failure_robustness(road_snapped_network_graph, max_path_length):
    return float(max_path_length) / float(len(road_snapped_network_graph) - 1)

def compute_network_statistics(road_snapped_network_graph):
    path_lengths = get_path_lengths(road_snapped_network_graph) # Get the sum of all possible
    avg_path_length = np.mean(path_lengths)
    max_path_length = max(path_lengths)

    return max_path_length, avg_path_length

def get_node_with_degree(node_degrees, degree):
    # Iterate over the DegreeView object to find the node with the specified degree
    for node, _ in node_degrees:
        if _ == degree:
            return node
    return None  # Return None if no node with the specified degree is found

def get_path_lengths(snapped_road_network_graph):
    return [sum(nx.single_source_shortest_path_length(snapped_road_network_graph, n).values())
            for n in snapped_road_network_graph]

### Network Analysis Metrics

In [85]:
# Longest route, shortest route, average route length, network diamater

### Visualizations

In [86]:
# This is to visualize the stops
def plot_stops_on_map(stops):
    # Iterate over the nodes in the network
    map_center = (14.599512, 120.984222)
    network_map = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')
    for stop in stops:
        
        if stop.isTranspo:
            marker_color = 'blue'
        else:
            marker_color = 'red'
            
        folium.Marker(location=[stop.lat, stop.long], popup=f"Transportation: {stop.isTranspo}", icon=folium.Icon(color=marker_color)).add_to(network_map)
        
    return network_map

In [87]:
# Test Manila Map

# Create a map centered at the initial location
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')
for route in manila_route_stops:
    for stop in route:
        folium.Marker(location=[stop[0], stop[1]], popup=f"{stop[0]}, {stop[1]}", icon=folium.Icon(color='blue')).add_to(m)
        
m.save('test_manila.html')

In [88]:
# Test Makati Map

# Create a map centered at the initial location
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')
for route in makati_route_stops:
    for stop in route:
        folium.Marker(location=[stop[0], stop[1]], popup=f"{stop[0]}, {stop[1]}", icon=folium.Icon(color='blue')).add_to(m)
        
m.save('test_makati.html')

In [89]:
# Test Mandaluyong Map

# Create a map centered at the initial location
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')
for route in mandaluyong_route_stops:
    for stop in route:
        folium.Marker(location=[stop[0], stop[1]], popup=f"{stop[0]}, {stop[1]}", icon=folium.Icon(color='blue')).add_to(m)
        
m.save('test_mandaluyong.html')

## Main

### Creating the Stops

In [91]:
# File Paths
pikl_filepath = "Original Jeepney Routes/Saved Networks/"
map_filepath = "Original Jeepney Routes/Maps/"

In [100]:
# Remove all points that are out of bounds
manila_route_stops_new = remove_if_out_of_bounds(MANILA_GRAPH, manila_route_stops)

REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED
REMOVED


In [103]:
transpo_points_Manila, non_transpo_points_Manila, route_list_Manila = check_transpo_stops(manila_route_stops_new, manila_points_gdf, MANILA_GRAPH, manila_idx)

In [104]:
# MANILA STOPCANDIDATES (METRO MANILA)
graph_of_stops_Manila = copy.deepcopy(METROMANILA_GRAPH)
list_of_stops_Manila = []

place_stops_on_roads(transpo_points_Manila, non_transpo_points_Manila, graph_of_stops_Manila, list_of_stops_Manila)

# Visualize the stops
map = plot_stops_on_map(list_of_stops_Manila)
map.save("stop_map_manila.html")

In [ ]:
# MANILA STOPCANDIDATES
graph_of_stops_Manila = copy.deepcopy(MANILA_GRAPH)
list_of_stops_Manila = []

place_stops_on_roads(transpo_points_Manila, non_transpo_points_Manila, graph_of_stops_Manila, list_of_stops_Manila)

# Visualize the stops
map = plot_stops_on_map(list_of_stops_Manila)
map.save("stop_map_manila.html")

In [ ]:
#Export stops to pickle
file_path = f'{pikl_filepath}stop_list_Manila.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(list_of_stops_Manila, f)
    
file_path = f'{pikl_filepath}stop_graph_Manila.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(graph_of_stops_Manila, f)

In [ ]:
# MAKATI STOPCANDIDATES
list_of_stops_Makati, stop_objs_route_Makati = check_stops(makati_route_stops, makati_points_gdf, MAKATI_GRAPH, makati_idx)

map = plot_stops_on_map(list_of_stops_Makati)
map.save("stop_map_orig_makati.html")

In [ ]:
# MANDALUYONG STOPCANDIDATES
list_of_stops_Mandaluyong, stop_objs_route_Mandaluyong = check_stops(mandaluyong_route_stops, mandaluyong_points_gdf, MANDALUYONG_GRAPH, mandaluyong_idx)

map = plot_stops_on_map(list_of_stops_Mandaluyong)
map.save("stop_map_orig_mandaluyong.html")

### Creating the routes

In [ ]:
route_network_Manila, route_network_graph_Manila = generate_route_network(stop_objs_route_Manila, MANILA_GRAPH)
Manila_Route_Network = networkObj(route_network_Manila, list_of_stops_Manila, route_network_graph_Manila)

In [ ]:
route_network_Makati, route_network_graph_Makati = generate_route_network(stop_objs_route_Makati, MAKATI_GRAPH)
Makati_Route_Network = networkObj(route_network_Makati, list_of_stops_Makati, route_network_graph_Makati)

In [ ]:
route_network_Mandaluyong, route_network_graph_Mandaluyong = generate_route_network(stop_objs_route_Mandaluyong, MANDALUYONG_GRAPH)
Mandaluyong_Route_Network = networkObj(route_network_Mandaluyong, list_of_stops_Mandaluyong, route_network_graph_Mandaluyong)

In [ ]:
for stop in list_of_stops_Makati:
    print(stop.id)

In [ ]:
for route in route_network_Mandaluyong:
    for path in route:
        print(len(path))

In [ ]:
len(route_network_graph_Mandaluyong)

In [ ]:
# Creating Maps for visualization

i = 1
for route_network in list_of_networks_Manila:
    map_center = (14.599512, 120.984222)
    m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

    # Plotting in the Map
    add_markers(route_network.stops, route_network.graph)
        
    for route in route_network.routes:
        for connection in route.route:
            ox.plot_route_folium(graph_of_stops_Manila, connection, route_map=m, tiles='openstreetmap', route_color="green")

    m.save(f"{map_html_location}Route Map-{i}.html")
    i += 1

#### Genetic Algorithm (Only to generate new GA results)

In [ ]:
# Evaluate the fitness of each network in the population
num_failure_removal = 4
weight_random_failure = 0.15
weight_targeted_failure = 0.15
weight_connectivity = 0.7

Manila_Route_Network.fitness_score = compute_fitness_score(Manila_Route_Network.graph, num_failure_removal, weight_random_failure, weight_targeted_failure, weight_connectivity)
print(f"Manila Route Network Score {Manila_Route_Network.fitness_score}")

Makati_Route_Network.fitness_score = compute_fitness_score(Makati_Route_Network.graph, num_failure_removal, weight_random_failure, weight_targeted_failure, weight_connectivity)
print(f"Makati Route Network Score {Makati_Route_Network.fitness_score}")

Mandaluyong_Route_Network.fitness_score = compute_fitness_score(Mandaluyong_Route_Network.graph, num_failure_removal, weight_random_failure, weight_targeted_failure, weight_connectivity)
print(f"Mandaluyong Route Network Score {Mandaluyong_Route_Network.fitness_score}")
